# Лабораторная работа 1. Нейронные сети

Результат лабораторной работы − отчет в формате Jupyter notebook'а. Постарайтесь сделать ваш отчет интересным рассказом, последовательно отвечающим на вопросы из заданий. Помимо ответов на вопросы, в отчете так же должен быть код, однако чем меньше кода, тем лучше всем: мне − меньше проверять, вам —  проще найти ошибку или дополнить эксперимент. При проверке оценивается четкость ответов на вопросы, аккуратность отчета и кода.

## Оценивание и штрафы

* Каждая из задач имеет определенную «стоимость» (указана в скобках около задачи)
* Максимально допустимая оценка за работу — 10 баллов
* При сдаче до soft deadline можно получить максимум 100% баллов
* При просрочке не более, чем на 5 минут — максимум 90% баллов
* При просрочке не более, чем на 1 час — максимум 80% баллов
* При сдаче до hard deadline — максиму 70% баллов
* Сдавать задание после hard deadline нельзя
* «Похожие» решения считаются плагиатом и все задействованные студенты (в том числе те, у кого списали) не могут получить за него больше 0 баллов и понижают карму
* Если вы нашли решение какого-то из заданий в открытом источнике, необходимо прислать ссылку на этот источник (скорее всего вы будете не единственным, кто это нашел, поэтому чтобы исключить подозрение в плагиате, необходима ссылка на источник)
* Не оцениваются задания с удалёнными формулировкам
* Не оценивается лабораторная работа целиком, если она была выложена в открытый источник

## Часть 1. Fine-tuning обученных нейросетей (6 баллов)

В этой части задания вам предстоит поработать с настоящими монстрами: сетями с почти сотней слоёв и десятками миллионов параметров. Например, такими:

![img](https://alexisbcook.github.io/assets/inception.png)
<center>googlenet inception v3</center>

Если внимателно всмотреться в картинку, можно заметить, что синим цветом обозначены свёрточные слои, красным — pooling, зелёным — конкатенация входов и т.п.

__Чем кормить такого монстра?__

Огромные нейросети обучаются на огромных массивах данных. В компьютерном зрении таких несколько, но самый популярный из них [ImageNet](http://image-net.org/). В этой выборке более миллиона изображений.

Задача этой сети состоит в классификации каждого изображение в один из 1000 классов. Вот они:

In [2]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import pickle

!wget https://www.dropbox.com/s/qy18cc1pl1gy6pn/classes.pkl

classes = pickle.load(open('classes.pkl','rb'))
print(classes[::100])

--2020-09-24 20:07:48--  https://www.dropbox.com/s/qy18cc1pl1gy6pn/classes.pkl
Resolving www.dropbox.com (www.dropbox.com)... 2620:100:6027:1::a27d:4801, 162.125.72.1
Connecting to www.dropbox.com (www.dropbox.com)|2620:100:6027:1::a27d:4801|:443... failed: Network is unreachable.
Connecting to www.dropbox.com (www.dropbox.com)|162.125.72.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/qy18cc1pl1gy6pn/classes.pkl [following]
--2020-09-24 20:07:49--  https://www.dropbox.com/s/raw/qy18cc1pl1gy6pn/classes.pkl
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucb3cd5e76d1f6a645bf3771c4e5.dl.dropboxusercontent.com/cd/0/inline/BAA40FwcF8XLblN8qge5ollGrn1DHSYYByzDyxT3frAFO81ESTiAUZU5gY0NCx4CRLJeHt6GlmH_p232NdunBTsMOOTMCJPTIUFUK6emg8Gs9VHgepAJJlsb_WYTjIDKFZ0/file# [following]
--2020-09-24 20:07:49--  https://ucb3cd5e76d1f6a645bf3771c4e5.dl.dropboxusercontent.com/cd/0/inline

### Зоопарк нейросетей в PyTorch

В PyTorch кроме всего прочего, есть зоопарк предобученных нейросетей. Так например сети, связанные с компьютерным зрением, находятся в [`torchvision.models`](https://pytorch.org/docs/stable/torchvision/models.html).

Ниже пример кода, который загружает обученную модель с картинки выше.

#### Внимание
[InceptionV3](https://pytorch.org/docs/stable/torchvision/models.html#inception-v3) требует много памяти для работы. Если ваш ПК начинает зависать,
* закройте всё кроме jupyter и браузера с одной вкладкой;
* если не помогло, загрузите эту тетрадку в [google colab](https://colab.research.google.com/) и работайте там.

In [3]:
from torchvision.models import inception_v3

In [29]:
model = inception_v3(pretrained=True)
model.eval()
model

Downloading: "https://download.pytorch.org/models/inception_v3_google-1a9a5a14.pth" to /home/vitonka/.torch/models/inception_v3_google-1a9a5a14.pth
100%|██████████| 108857766/108857766 [00:09<00:00, 11548395.26it/s]


Inception3(
  (Conv2d_1a_3x3): BasicConv2d(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2a_3x3): BasicConv2d(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2b_3x3): BasicConv2d(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_3b_1x1): BasicConv2d(
    (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_4a_3x3): BasicConv2d(
    (conv): Conv2d(80, 192, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, t

Реализуем функцию, которая применяет готовую модель и выдаёт топ-10 предположений в порядке убывания вероятностей:

In [64]:
import torch
from torchvision import datasets, transforms as T


def predict_top10(img):
    normalize = T.Normalize(mean=[0.485, 0.456, 0.406],
                            std=[0.229, 0.224, 0.225])
    transform = T.Compose([T.Resize((299, 299)), T.ToTensor(), normalize])
    probs = model(transform(img).unsqueeze(0))
    labels = probs.argsort()[0].numpy()[-1:-10:-1]
    
    return [classes[label] for label in labels]

И проверим её на картинке с альбатросом!

In [8]:
!wget https://avatars.mds.yandex.net/get-pdb/1792436/cd9c45cc-037b-4255-96ad-8e4ca42e1739/s1200 -O albatross.jpg

--2020-09-24 20:18:55--  https://avatars.mds.yandex.net/get-pdb/1792436/cd9c45cc-037b-4255-96ad-8e4ca42e1739/s1200
Resolving avatars.mds.yandex.net (avatars.mds.yandex.net)... 2a02:6b8::184, 87.250.247.181, 87.250.247.184, ...
Connecting to avatars.mds.yandex.net (avatars.mds.yandex.net)|2a02:6b8::184|:443... failed: Network is unreachable.
Connecting to avatars.mds.yandex.net (avatars.mds.yandex.net)|87.250.247.181|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 85638 (84K) [image/jpeg]
Saving to: ‘albatross.jpg’

albatross.jpg       100%[===================>]  83,63K  --.-KB/s    in 0,04s   

2020-09-24 20:18:56 (2,21 MB/s) - ‘albatross.jpg’ saved [85638/85638]



In [9]:
from PIL import Image

img = Image.open('albatross.jpg')

In [65]:
predict_top10(img)

['albatross, mollymawk',
 'kite',
 'turnstile',
 'breakwater, groin, groyne, mole, bulwark, seawall, jetty',
 'oystercatcher, oyster catcher',
 'goose',
 'desktop computer',
 'Polaroid camera, Polaroid Land camera',
 'cinema, movie theater, movie theatre, movie house, picture palace']

Как видно, даже из коробки работает неплохо!

### Dogs Vs Cats

А теперь попробуем построить классификатор, который отличает изображение кошки от собаки. 

![img](https://dingo.care2.com/pictures/greenliving/1203/1202163.large.jpg)

In [66]:
!wget https://www.dropbox.com/s/7qhtkw49tyog56u/data.zip -O data.zip
!unzip data.zip

--2020-09-24 20:35:51--  https://www.dropbox.com/s/7qhtkw49tyog56u/data.zip
Resolving www.dropbox.com (www.dropbox.com)... 2620:100:6027:1::a27d:4801, 162.125.72.1
Connecting to www.dropbox.com (www.dropbox.com)|2620:100:6027:1::a27d:4801|:443... failed: Network is unreachable.
Connecting to www.dropbox.com (www.dropbox.com)|162.125.72.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/7qhtkw49tyog56u/data.zip [following]
--2020-09-24 20:35:53--  https://www.dropbox.com/s/raw/7qhtkw49tyog56u/data.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc6ed6e926adca64ad0c9b101cc7.dl.dropboxusercontent.com/cd/0/inline/BACXl4SVMpiKyf0TI-xOaFcL20KLvzSBs9Z3yZZ9wohAYFBpCRQs9nZKVWkt9Vf8Jtc4Zvi3WTMP7v6WqXrqsV76qzPe6C-AKI05i7hqbB5MS5Kf3gUUkxZQW16Py3120KA/file# [following]
--2020-09-24 20:35:53--  https://uc6ed6e926adca64ad0c9b101cc7.dl.dropboxusercontent.com/cd/0/inline/BACXl4SV

### sklearn way

В вашем распоряжении есть предобученная сеть InceptionV3. Ваша задача — обучить классификатор из sklearn (на ваш выбор), который будет отличать котов от собак, используя __активации нейронной сети в качестве признаков__.

Для начала, прочитайте данные и сформируйте для вашего классификатора обучающую и тестовую выборки в пропорции 4:1. 

В вашем распоряжении всего 25 000 изображений различного размера, все в формате JPEG. Все картинки лежат в папке __`./train`__. Изображения кошек имеют название вида `./train/cat.*.jpg`, собак — `./train/dog.*.jpg`.

In [67]:
# разделите данные на обучение и тест в отношении 20k/5k

# ВАШ КОД

**Задание 1 (0.5 балла)**. Для каждой картинки вычислите признаки из промежуточного слоя свёрточной сети. В качестве признаком можно выбрать какой-нибудь слой или несколько слоёв сети. Попробуйте найти комбинацию слоёв, которая работает лучше всего.

In [ ]:
# прочитайте данные и вычислите признаки

# ВАШ КОД

**Задние 2 (1 балл)**. Обучите поверх этих признаков классификатор из sklearn (можно попробовать несколько и выбрать лучший). Попробуйте получить AUC __хотя бы 99%__.

In [ ]:
# обучите и примените классификаторы из sklearn. выберите лучшую модель по AUC.

# ВАШ КОД

**Задание 3 (1 балл)**. Напишите ход ваших исследований, а также выводы о проделанной работе.

### Fine-tuning

Давайте попробуем добиться ещё большего качество через дообучение (fine-tuning) модели. Новая цель — получить качество лучше, чем у логистической регрессии. Вероятность ошибки - __менее 0.5%__.

**Задание 4 (1 балл)**.

__Шаг 1.__ - постройте сеть, в которой InceptionV3 "без головы" используется в качестве первого слоя. Поверх неё соорудите новую голову - она будет отличать котов от собак.

__Шаг 2.__ - обучите "голову" на обучающей выборке. При этом не изменяйте веса изначальной сети.

__Sanity check:__ После этого шага ваша модель должна уже быть сравнима по точности с моделями из задания 1.

Если всё получилось, самое время сохранить модель.

**Задание 5 (1 балл)**.

__Шаг 3.__ "Разморозьте" несколько предыдущих слоёв модели и продолжите обучение. На этом этапе важно не переобучиться: смотрите на качество на валидации.

Если качество не улучшается, а сразу идёт вниз, попробуйте уменьшить число обучаемых слоёв или воспользуйтесь аугментацией данных. In fact, попробуйте аугментацию данных даже если и без неё всё работает - иногда она творит чудеса.

__Шаг 4.__ Вычислите финальное качество. Напомню, ошибка должна составлять менее 0.5%.

**Задание 6 (1.5 балла)**. Напишите отчёт и вознаградите себя за старания чем-нибудь. А я награжу вас баллами.

## Часть 2. Делаем reid c нейронками (4 балла)

[Описание задачи](https://www.kaggle.com/c/pedestrian-reid-yandex-shad-fall-2020)

In [4]:
from pathlib import Path

import torch

DEVICE = 'cpu'
if torch.cuda.is_available():
    DEVICE = 'cuda'

IMAGES_PATH = Path('train')

### Посмотрим на наши данные

In [5]:
import pandas as pd
meta = pd.read_csv('train.csv')
meta.head()

FileNotFoundError: [Errno 2] File b'train.csv' does not exist: b'train.csv'

In [ ]:
from PIL import Image
from random import randint

row = meta.iloc[randint(0, len(meta))]

Image.open(IMAGES_PATH / (str(row.image_id )+ '.png'))

**Задача 7 (1 балл)** Допишите класс `ReIdDataset`, который можно будет использоваться в качестве аргумента в `torch.utils.data.DataLoader`. Класс может загружать картинки во время вызова `__getitem__`, или прочитать их все сразу в оперативную память в `__init__`. Для загрузки картинок рекомендуется использовать `PIL.Image` из пакета `pillow`.

[Пример](https://pytorch.org/tutorials/beginner/data_loading_tutorial.html) реализации кастомных датасетов.

In [ ]:
class ReIdDataset:
    def __init__(self, csv_path, images_path, transforms=None):
        self.transforms = transforms
        self.meta = pd.read_csv(csv_path)
        self.images_path = images_path
        
    def __len__(self):
        raise NotImplemented()
        
    def __getitem__(self, idx):
        
        return {
            'image_id': None,
            'image': None,
            'person_id': None,
        }

Сделаем `DataLoader`. Обратите внимание, что картинки идут в разных разрешениях, поэтому в батче их нужно будет привести к одному размеру, что можно сделать при помощи `torchvision.transforms.Resize`.


In [ ]:
from torch.utils.data import DataLoader
import torchvision.transforms as tt

transforms = tt.Compose([
    tt.Resize((224, 112)),
    tt.ToTensor(),
    tt.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

dataset = ReIdDataset('train.csv', IMAGES_PATH, transforms=transforms)

loader = DataLoader(dataset, batch_size=64, shuffle=True, num_workers=7)

Код обучения модели уже сделан за вас в целях демонстрации

In [ ]:
import time

import numpy as np

def train_epoch(model, criterion, optimizer, loader, device):
    model.train()
    model = model.to(device)
    start = time.time()
    for i, batch in enumerate(loader):
        images = batch['image'].to(device)
        labels = batch['person_id'].to(device)

        optimizer.zero_grad()
        predictions = model(images)

        loss = criterion(predictions['classes'], labels)
        loss.backward()
        optimizer.step()

        accuracy = (predictions['classes'].max(-1)[-1] == labels).to('cpu').detach().numpy().mean()
        
        print(f'batch {i+1}/{len(loader)} | loss={loss.item():.3f} | acc={accuracy:.2f}', end='\r' if i+1!=len(loader) else '\n')

    end = time.time()
    print(f'elapsed {end - start:0.0f} seconds')

В качестве backbone для baseline решения используется архитектура [resnet34](https://github.com/pytorch/vision/blob/a4736ea6e8ff25a1265ca5adf9e6e244d78500e8/torchvision/models/resnet.py#L244). Обратите внимание на флаг `pretrained`: если он включен, то загрузятся веса модели, обученной на датасете [ImageNet](http://www.image-net.org/). Это позволит нам сразу использовать "хорошие" признаки, поверх которых мы обучим модель re-id.

**Задача 8 (1 балл)** Реализуйте метод `get_embeddings` -- должен возвращать признаки, которые идут на вход слоя `resnet34.fc` (смотри `ResNet._forward_impl` по ссылке выше). 


In [ ]:
from torchvision.models.resnet import resnet34

class Model(torch.nn.Module):
    def __init__(self, n_classes):
        super().__init__()
        self.backbone = resnet34(pretrained=True, progress=False)
        self.classifier = torch.nn.Linear(512, n_classes, bias=False)
        
    def get_embeddings(self, x):
        raise NotImplemented()
        
    def forward(self, x):
        embeddings = self.get_embeddings(x)
        classes = self.classifier(embeddings)
        return {
            'embeddings': embeddings,
            'classes': classes,
        }

In [ ]:
model = Model(n_classes=len(meta.person_id.unique())).to(DEVICE)

**Задача 9 (1 балл)** Fine tuning. Так как наш датасет очень мал (~10_000 картинок), то тюнинг всех весов модели может привести к переобучению. Более того, если случайно проинициализировать веса последнего полносвзяного слоя, то получится градиент с большой нормой, который "выбросит" остальные веса модели из области с "хорошими" признаками. Поэтому рекомендуется сначала обучить просто классификатор поверх "замороженного" backbon'а. Ключевое слово -- `requires_grad`

In [ ]:
# заморозьте все веса модели
# разморозьте веса классификатора людей

Обучим веса классификатора:

In [ ]:
from torch.optim import Adam
optimizer = Adam(model.parameters(), lr=0.0015, amsgrad=True)
criterion = torch.nn.CrossEntropyLoss()

In [ ]:
for i in range(2):
    print(f'EPOCH {i+1}')
    train_epoch(model, criterion, optimizer, loader, device=DEVICE)

Теперь можно начать обучать остальные веса. В условиях маленького датасета рекомендуется не трогать ранние слои, поэтому попробуем обучить только `model.backbone.layer4` (имя слоя можно подсмотреть в коде модели).

In [ ]:
# разморозьте model.backbone.layer4

Теперь обучим вместе классификатор и `model.backbone.layer4`

In [ ]:
for i in range(5):
    print(f'EPOCH {i+1}')
    train_epoch(model, criterion, optimizer, loader, device=DEVICE)

Пришло время делать сабмит

In [ ]:
test_dataset = ReIdDataset('test.csv', 'test', transforms=transforms)

test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=7)

Так как множества людей в обучающих и тестовых данных не пересекаются, то слой классификатора нам бесполезен. Однако, в силу того, как мы обучали нашу модель, мы можем использовать наши эмбединги, чтобы сравнивать между собой людей. Сравнивать их можно по косиносному расстоянию, так как учили cross-entropy поверх softmax (подумайте, почему).

In [ ]:
def get_features(model, loader, device):
    model.eval()
    features, images_ids = [], []
    for batch in loader:
        batch_features = model(batch['image'].to(device))['embeddings']
        features.extend(batch_features.to('cpu').data.numpy())
        images_ids.extend(batch['image_id'])
    features = np.array(features)
    images_ids = np.array(images_ids)
    return features, images_ids

In [ ]:
features, images_ids = get_features(model, test_loader, device=DEVICE)

**Задача 10 (1 балл)** Реализуйте функцию подсчета *расстояния* между всеми парами эмбеддингов.

In [ ]:
def cos_dist(features):
    raise NotImplemented()

In [ ]:
# Так как считается MAP@20, то нам нужно для каждого человека взять 20 его ближайших соседей.
# Самый ближайший сосед -- это сам человек, но в сабмит его отправлять не нужно, поэтому берем первые 21 соседей.
order = np.argsort(cos_dist(features))[:, :21]
submit = np.take(images_ids, order)

Осталось сгенерировать сам сабмит...

In [ ]:
import csv
with open('baseline', 'w') as f:
    writer = csv.DictWriter(f, fieldnames=['query', 'retrieval'])
    writer.writeheader()
    for row in submit:
        writer.writerow({
            'query': row[0],
            'retrieval': ' '.join(row[1:])
        })

# Что рекомендуется сделать

1. Локальную валидацию
2. [Аугментации](https://pytorch.org/docs/stable/torchvision/transforms.html): `torchvision.transforms.{ColorJitter,Random*}`
3. [Dropout](https://pytorch.org/docs/stable/generated/torch.nn.Dropout.html) перед слоем классификатора
4. Обучить больше эпох -- особенно актуально, если используются регуляризации и аугментации
5. [Re-ranking](https://arxiv.org/abs/1701.08398)
6. [Am-softmax](https://arxiv.org/pdf/1801.05599.pdf). Обратите внимание на активацию relu перед слоем эмбеддингов
7. Попробуйте разные архитектуры для fine tuning'а: [torchvision.models](https://pytorch.org/docs/stable/torchvision/models.html), [EfficientNet](https://github.com/lukemelas/EfficientNet-PyTorch).
8. Последние эпохи учить модель с меньшим шагом обучения: https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate (попробуйте [MultiStepLR](https://pytorch.org/docs/stable/optim.html#torch.optim.lr_scheduler.MultiStepLR) или [CosineAnnealingLR](https://pytorch.org/docs/stable/optim.html#torch.optim.lr_scheduler.CosineAnnealingLR)).
9. Почитать статьи с трюками для обучения [нейронок](https://arxiv.org/abs/1812.01187)/[reid](https://openaccess.thecvf.com/content_CVPRW_2019/papers/TRMTMCT/Luo_Bag_of_Tricks_and_a_Strong_Baseline_for_Deep_Person_CVPRW_2019_paper.pdf).